# 機械学習ハンズオン（ワークフロー編）

## 1. ハンズオンの概要

[UCIのAdultデータセット](https://archive.ics.uci.edu/ml/datasets/Adult)を使って、
年齢や職業などのデータから、その人の収入が5万ドル以上あるかどうかの2値分類(binary classification)を行います。

このハンズオンの流れは次のとおりです。
 1. データの取得
 1. データの分析
 1. データの前処理
 1. 学習モデルの作成
 1. 学習モデルの評価


## 2. 事前準備

### 2.1. ランタイムの確認

Google Colabを使っている場合は、メニューから「ランタイム」→「ランタイムのタイプを変更」を選択して、「ハードウェア アクセラレータ」を「GPU」に設定してください。


### 2.2. ライブラリのロード

In [0]:
!pip install pandas tensorflow numpy matplotlib seaborn scikit-learn
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## 3. データ取得

pandasのAPIを使ってデータファイルを読み込みます。
 * このファイルにはヘッダー行がないので、ヘッダーは自分で設定します。
 * 不明値を表す "?" はN/Aに変換しておきます。
   * のちほど不明値を処理します。

In [0]:
headers = ('age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income')
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', sep=', ', names=headers, na_values='?')

読み込んだデータを表示してみましょう。

In [0]:
df.head(10)

## 4. データ分析

### 4.1. ラベルごとのデータ件数

5万ドル未満が約76%あるので、**学習モデルがすべて5万ドル未満と予測しても76%前後の正答率が出てしまう**ことに注意が必要です。

In [0]:
df.groupby('income').size()

In [0]:
df.groupby('income').size() / len(df)

### 4.2. 量的変数の分析

#### ラベル別の特徴量の分布
fntwgtは、5万ドル超も5万ドル以下も同じ分布を取っているので、ラベルとは無関係と判断し、特徴量から除外することにします。

In [0]:
plt.figure(figsize=(20, 10))
features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
for i in range(len(features)):
  plt.subplot(2, 3, i+1)
  plt.title(features[i])
  sns.kdeplot(df[df['income']=='<=50K'][features[i]], label='<=50K')
  sns.kdeplot(df[df['income']=='>50K'][features[i]], label='>50K')


In [0]:
df[df['income']=='<=50K'].describe()

In [0]:
df[df['income']=='>50K'].describe()

### 4.3. 質的変数の分析

#### ラベル別の特徴量の分布

In [0]:
pd.crosstab(index=df['income'], columns=df['workclass'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['marital-status'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['occupation'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['relationship'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['race'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['sex'], normalize='columns')

In [0]:
pd.crosstab(index=df['income'], columns=df['native-country'], normalize='columns')

### 4.4. 特徴量間の関係の分析

#### "education" vs "education-num"
下図から同値だと判断できるため、"education"は除外することにします。

In [0]:
sns.boxplot(y='education', x='education-num', data=df)

### 4.5. 欠損値のチェック

欠損値があるので、あとでこのレコードを削除します。

In [0]:
df.isnull().sum()

## 5. データ前処理

### 5.1. 前処理をする前の状態

In [0]:
df.head(10)

### 5.2. 欠損値のあるレコードの削除

In [0]:
df = df.dropna()
df.isnull().sum()

### 5.3. ラベルの作成
データから"income"列を抜き出し、"<=50K", ">50K"をそれぞれ0, 1の数値に変換します。

In [0]:
ys = pd.get_dummies(df['income'], drop_first=True)
ys.head(10)

### 5.4. 不要な特徴量の削除
 * "income"はラベルなので削除
 * "fnlwgt"はラベルと相関がないので削除
 * "education"は"education-num"と同一の特徴なので削除

In [0]:
drop_columns = ['income', 'fnlwgt', 'education']
df = df.drop(drop_columns, axis=1)
df.head(10)

### 5.5. 質的変数のダミー化

In [0]:
xs = pd.get_dummies(df)
xs.head(10)

## 6. 学習モデルの作成

### 6.1. データ分割
データを訓練データ・検証データ・テストデータの3つに分割します。

まず、全体の20%をテストデータに回し、残ったデータの20%を検証データに回します。

In [0]:
all_xs = xs.values
all_ys = ys.values
tmp_xs, test_xs, tmp_ys, test_ys = train_test_split(all_xs, all_ys, test_size=0.2)
train_xs, valid_xs, train_ys, valid_ys = train_test_split(tmp_xs, tmp_ys, test_size=0.2)
print(train_xs.shape, valid_xs.shape, test_xs.shape, train_ys.shape, valid_ys.shape, test_ys.shape)

### 6.2. 正規化
特徴量を $0.0\le{}x\le{}1.0$ の範囲に収まるように正規化します。

In [0]:
scaler = MinMaxScaler()
scaler.fit(all_xs)
train_xs = scaler.transform(train_xs)
valid_xs = scaler.transform(valid_xs)
test_xs = scaler.transform(test_xs)

### 6.3. 学習モデル構築

まずは単層のパーセプトロンモデルを作りましょう。
 * パラメータ数は特徴量の数と同じ
 * 出力次元数は2値分類なので1

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, input_dim=train_xs.shape[1], activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

## 7. 学習モデルの評価

### 7.1. 学習実行

実際に学習
エポック（すべての訓練データを1回学習させることを**1エポック**と呼びます）ごとに、訓練データ・検証データそれぞれに対する損失・正答率が出力されます。
 * `loss` : 訓練データの損失
 * `acc` : 訓練データの正答率
 * `val_loss`: 検証データの損失
 * `val_acc`: 検証データの正答率

In [0]:
hist = model.fit(train_xs, train_ys, batch_size=128, epochs=100, validation_data=(valid_xs, valid_ys))

### 7.2. モデルの評価
訓練データ・学習データに対する損失と正答率をグラフ化してみましょう。

In [0]:
%matplotlib inline
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, 101), hist.history["loss"])
plt.plot(range(1, 101), hist.history["val_loss"])
plt.title("loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.subplot(1, 2, 2)
plt.plot(range(1, 101), hist.history["acc"])
plt.plot(range(1, 101), hist.history["val_acc"])
plt.title("accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")

テストデータに対する性能を求めてみましょう。

In [0]:
pred = model.predict_classes(test_xs, batch_size=128)
accuracy = accuracy_score(test_ys, pred)
precision = precision_score(test_ys, pred)
recall = recall_score(test_ys, pred)
f1 = f1_score(test_ys, pred)
print("accuracy = {:.2f}, precision = {:.2f}, recall = {:.2f}, F1-score = {:.2f}".format(accuracy, precision, recall, f1))

### 7.3. 別のモデルの評価
今度はパーセプトロンを3層にしたモデルを試してみましょう。

In [0]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_dim=train_xs.shape[1], activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [0]:
hist = model.fit(train_xs, train_ys, batch_size=128, epochs=100, validation_data=(valid_xs, valid_ys))

In [0]:
%matplotlib inline
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, 101), hist.history["loss"])
plt.plot(range(1, 101), hist.history["val_loss"])
plt.title("loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.subplot(1, 2, 2)
plt.plot(range(1, 101), hist.history["acc"])
plt.plot(range(1, 101), hist.history["val_acc"])
plt.title("accuracy")
plt.xlabel("epoch")
plt.ylabel("accuracy")

In [0]:
pred = model.predict_classes(test_xs, batch_size=128)
accuracy = accuracy_score(test_ys, pred)
precision = precision_score(test_ys, pred)
recall = recall_score(test_ys, pred)
f1 = f1_score(test_ys, pred)
print("accuracy = {:.2f}, precision = {:.2f}, recall = {:.2f}, F1-score = {:.2f}".format(accuracy, precision, recall, f1))

## 8. 課題

 1. 特徴量を年齢("age")だけにしたときにどの程度の性能が出るか、試してみましょう。
 1. 層数や各層のノード数を変えたモデルを作って試してみましょう。

## 9. さいごに

Kaggleに[このデータセットのカーネル](https://www.kaggle.com/uciml/adult-census-income/kernels)が数多く公開されています。
これを見て、ほかの人がどのようにこのデータセットの分析を行っているのかを勉強してみてください。